In [ ]:
let buildDir = "./bin/Debug/net8.0" 
let copyDir = "./bin/Debug/notebook"

// Clear existing copy directory
if System.IO.Directory.Exists(copyDir) then 
    System.IO.Directory.Delete(copyDir, true)

let rec copyBuildDir (source : string) (target: string) =
    System.IO.Directory.CreateDirectory(target) |> ignore  
    for file in System.IO.Directory.GetFiles(source) do 
        File.Copy(file, System.IO.Path.Combine(target, System.IO.Path.GetFileName(file)))

    for dir in System.IO.Directory.GetDirectories(source) do
        let newDir = System.IO.Path.Combine(target, System.IO.Path.GetFileName(dir)) 
        copyBuildDir dir newDir

copyBuildDir buildDir copyDir

In [ ]:
#r "./bin/Debug/notebook/studies.dll"
#r "./bin/Debug/notebook/core.fs.dll"

#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

In [ ]:
open FSharp.Data
open Plotly.NET
open core.fs.Stocks

let outputToConsole (message:string) =
        System.Console.WriteLine(message)

In [ ]:
let studiesDirectory = "d:\\studies"
let inputFilename = "d:\\studies\\signals_transformed.csv"

let priceFunc ticker = studies.DataHelpers.getPricesFromCsv studiesDirectory ticker

let inputSignals =
    inputFilename
    |> studies.Types.SignalWithPriceProperties.Load
    |> _.Rows
    |> Seq.toList
    
$"Loaded {inputSignals.Length} signals"

In [ ]:
// use this cell for signal filtering
let isInDownCycle (row:studies.Types.SignalWithPriceProperties.Row) =
    match row.Date with
    | x when x >= "2022-04-22" && x <= "2022-05-25" -> true
    | x when x >= "2022-06-10" && x <= "2022-07-07" -> true
    | x when x >= "2022-08-19" && x <= "2022-10-17" -> true
    | x when x >= "2022-12-20" && x <= "2023-01-06" -> true
    | x when x >= "2023-02-16" && x <= "2023-03-28" -> true
    | x when x >= "2023-05-02" && x <= "2023-06-02" -> true
    | x when x >= "2023-08-02" && x <= "2023-11-02" -> true
    | x when x >= "2024-01-16" && x <= "2024-07-07" -> true
    | _ -> false
    
let isInUpCycle x = not <| isInDownCycle x

let filter, cycleType = isInUpCycle, "UP cycle"

outputToConsole $"filtering out by cycle, keeping only things that are {cycleType}"

let signals = inputSignals |> List.filter filter

outputToConsole $"after filtering, we have {signals.Length} signals"
    

In [ ]:
let strategies = [
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long None None   // no stop, buy and hold or sell and hold
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short None None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 30) None // no stop, but only hold for 30 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 30) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 60) None // no stop, but only hold for 60 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 60) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long None (Some 0.05m) // 5% stop
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short None (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 30) (Some 0.05m) // 5% stop plus holding at most 30 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 30) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 60) (Some 0.05m) // 5% stop plus holding at most 60 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 60) (Some 0.05m)
    studies.Trading.strategyWithSignalOpenAsStop false  // signal open as stop
    studies.Trading.strategyWithSignalCloseAsStop false // signal close as stop
    studies.Trading.strategyWithTrailingStop false StockPositionType.Long 0.1m  // trailing stop of 10%
    studies.Trading.strategyWithTrailingStop false StockPositionType.Short 0.1m
]

let outcomes = 
    strategies
    |> studies.Trading.runTrades priceFunc signals
    |> Async.RunSynchronously
    |> Seq.toList

printfn "Finished running trades, generated:"
printfn $"{outcomes.Length} trade outcomes"

In [ ]:
// leave this cell for outcome checking from time to time when troubleshooting
//outcomes |> List.filter (fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28 && o.OpenPrice < o.Sma200 && o.Strategy = "Buy")

In [ ]:
let columnChart label keys values =
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
let histogramChart label bins values =
    let chart = 
        Chart.Histogram(
            X = values,
            NBinsX = bins,
            Name = label
        )
    chart.Display() |> ignore

let outputSummary outputFunc includeDistributionChart (summary:studies.Types.TradeSummary) = 

    // this ensures that the numbers align instead of negative EV taking up more space
    let evPositiveSign =
        match summary.EV with
        | x when x >= 0m -> "+"
        | _ -> ""
    
    let format = "0.00"
    let percentFormat = "00.00%"
    outputFunc $"{evPositiveSign}{summary.EV.ToString(percentFormat)} EV -> {summary.WinPct.ToString(percentFormat)} \t {summary.AvgWin.ToString(percentFormat)} \t {summary.AvgLoss.ToString(percentFormat)} \t {summary.AvgGainLoss.ToString(format)} avg gain/loss, \t {summary.TotalGain.ToString(format)} total gain, {summary.StrategyName} [{summary.NumberOfTrades} trades]"
        
    match includeDistributionChart with
    | true ->
        histogramChart "Gain Distribution" 40 (summary.Gains)
    | false -> ()
    
    
let genericFilters = [
    ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
    ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
    ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
    ("Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 30)
    ("New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 31)
]

let gapUpFilters = [
    ("Gap ups - New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Gap > 0m && o.Screenerid = 28)
    ("Gap ups - Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Gap > 0m && o.Screenerid = 29)
    ("Gap downs - Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Gap < 0m && o.Screenerid = 30)
    ("Gap downs - New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Gap < 0m && o.Screenerid = 31)
]

let sma20PriceFilters = [
    ("price>20 - New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Screenerid = 28)
    ("price>20 - Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Screenerid = 29)
    ("price>20 - Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Screenerid = 30)
    ("price>20 - New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Screenerid = 31)

    ("price<20 - New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Screenerid = 28)
    ("price<20 - Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Screenerid = 29)
    ("price<20 - Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Screenerid = 30)
    ("price<20 - New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Screenerid = 31)
]

let sma200PriceFilters = [
    ("price>200 - New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma200 && o.Screenerid = 28)
    ("price>200 - Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma200 && o.Screenerid = 29)
    ("price>200 - Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma200 && o.Screenerid = 30)
    ("price>200 - New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma200 && o.Screenerid = 31)

    ("price<200 - New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma200 && o.Screenerid = 28)
    ("price<200 - Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma200 && o.Screenerid = 29)
    ("price<200 - Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma200 && o.Screenerid = 30)
    ("price<200 - New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma200 && o.Screenerid = 31)
    
    
    ("price>smaAlign - New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid = 28)
    ("price>smaAlign - Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid = 29)
    ("price>smaAlign - Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid = 30)
    ("price>smaAlign - New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice > o.Sma20 && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid = 31)

    ("price<smaAlign - New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid = 28)
    ("price<smaAlign - Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid = 29)
    ("price<smaAlign - Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid = 30)
    ("price<smaAlign - New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.OpenPrice < o.Sma20 && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid = 31)
]

// filter out outcomes that have minimum of trades we are interested in
let minimumTradesPerCategory = 10

[<Flags>]
type FilterSelection =
    | GenericFilters = 1
    | Gaps = 2
    | Sma20 = 4
    | Sma200 = 8
    | All = 255

let createTradeSummaries filterSelection outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    let filterNamePairs = [
        if filterSelection &&& FilterSelection.GenericFilters = FilterSelection.GenericFilters then yield! genericFilters    
        if filterSelection &&& FilterSelection.Gaps = FilterSelection.Gaps then yield! gapUpFilters
        if filterSelection &&& FilterSelection.Sma20 = FilterSelection.Sma20 then yield! sma20PriceFilters
        if filterSelection &&& FilterSelection.Sma200 = FilterSelection.Sma200 then yield! sma200PriceFilters
    ]

    filterNamePairs
    |> Seq.collect (
        fun (filterName,filter) ->

            tradesGroupedByStrategy
            |> Seq.map (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    
                    let strategy = filteredTrades |> Seq.tryHead |> Option.map (fun x -> x.Strategy)
                    match strategy with
                    | None -> None
                    | Some strategy ->
                        let name = $"{filterName}, {strategy}"
                        let summary = filteredTrades |> studies.Types.TradeSummary.create name
                        match summary.NumberOfTrades with
                        | x when x < minimumTradesPerCategory -> None
                        | _ -> Some summary
            )
            |> Seq.choose id
    )
    

type OutcomeGrouping =
    | Signal
    | Strategy
    
let describeOutcomes outputFileOption outcomeGrouping includeEVChart includeDistributionChart tradeSummaries =
    
    let outputContent message =
        match outputFileOption with
        | None -> ()
        | Some outputFile -> System.IO.File.AppendAllText(outputFile, message + System.Environment.NewLine)
        System.Console.WriteLine(message)
        
    let groupingFunction =
        match outcomeGrouping with
        | OutcomeGrouping.Signal -> (fun (s:studies.Types.TradeSummary) -> s.StrategyName.Split(",")[0])
        | OutcomeGrouping.Strategy -> (fun (s:studies.Types.TradeSummary) -> s.StrategyName.Split(",")[1])
        
    let groupedData = tradeSummaries |> Seq.groupBy groupingFunction
    
    groupedData
    |> Seq.iter (
        fun (groupingName,summaries) ->
            
            outputContent $"{groupingName}"
            
            let sortedSummaries = summaries |> Seq.sortByDescending (fun summary -> summary.EV)
            
            sortedSummaries |> Seq.iter (fun summary -> outputSummary outputContent includeDistributionChart summary)
            
            if includeEVChart then
                let keys = sortedSummaries |> Seq.map (fun summary -> summary.StrategyName)
                let values = sortedSummaries |> Seq.map (fun summary -> summary.EV)
            
                columnChart groupingName keys values
                
            outputContent ""
            outputContent ""
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
        
    let sortFunctions = [
        ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
        ("Total Gain", fun (t:studies.Types.TradeSummary) -> t.TotalGain)
        ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
        ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        tradeSummaries
        |> Seq.sortByDescending sortFunction
        |> Seq.take numberOfTopRecords
        |> Seq.iter (fun summary -> summary |> outputSummary outputToConsole false)
        printfn ""
        printfn ""
    )

In [ ]:
printfn "Generating trade summaries..."
let summaryBreakdown = FilterSelection.All
let tradeSummaries = outcomes |> createTradeSummaries summaryBreakdown |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

tradeSummaries
|> describeOutcomes None OutcomeGrouping.Strategy false false

printfn "Finished"

In [ ]:
"Highlighting trades" |> outputToConsole

tradeSummaries
|> highlightStrategies 20

In [ ]:
let strategiesOfInterest = [
    "SL of"
    "as stop"
]

let summarizeSubset (name:string) =
    tradeSummaries
        |> Seq.filter (fun summary -> summary.StrategyName.Contains(name))
        |> Seq.sortByDescending (fun summary -> summary.EV)
        |> Seq.iter (fun summary -> 
            outputSummary outputToConsole false summary
        )
    
    printfn ""
    
strategiesOfInterest |> List.iter (fun n -> summarizeSubset n)
    

In [ ]:
// sanity check, print random trades, look over them, investigate specific trades if needed
let positionDescription (signal:studies.Types.TradeOutcomeOutput.Row) =
    // it will consist of components that describe signal
    // gap up or gap down
    // price>20, price<20, put together into a string

    let signalDescription =
        match signal.Screenerid with
        | 28 -> "New High"
        | 29 -> "Top Gainer"
        | 30 -> "Top Loser"
        | 31 -> "New Low"
        | _ -> failwith $"unknown signal: {signal.Screenerid}" 
    
    let gapDescription =
        match signal.Gap with
        | x when x > 0m -> "Gap Up"
        | x when x < 0m -> "Gap Down"
        | _ -> ""
        
    let priceDescription =
        match signal.OpenPrice with
        | x when x >= signal.Sma20 -> "price>20"
        | x when x < signal.Sma20 -> "price<20"
        | _ -> failwith "freakout"
        
    String.concat " " [signalDescription; gapDescription; priceDescription]
        
let outcomeDescription (x:studies.Types.TradeOutcomeOutput.Row) =
    let gain = x.PercentGain.ToString("##.##%")
    let signal = x |> positionDescription
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened} -> {x.Closed}, {x.OpenPrice} -> {x.ClosePrice}: profit of {gain}, signal: {signal}, strat: {x.Strategy}"
    
let random = System.Random()

let tradeStrategyOfInterest = "Sell"

//let filterFunc x = x |> signalDescription |> _.Contains(tradeStrategyOfInterest)
let filterFunc (x:studies.Types.TradeOutcomeOutput.Row) = x.Strategy = tradeStrategyOfInterest

outcomes
|> Seq.filter filterFunc
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription

printfn ""
printfn $"Max gain trade from {tradeStrategyOfInterest}"

outcomes
|> Seq.filter filterFunc
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

printfn ""
printfn "Max gain overall"

outcomes
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

In [ ]:
// investigating what happened with the trade and why it went as it went
let signals =
    inputFilename
    |> studies.Types.SignalWithPriceProperties.Load
    |> _.Rows

let tickerOfInterest = "MDGL"

let signalsOfInterest = 
    signals
    |> Seq.filter (fun signal -> signal.Ticker = tickerOfInterest)
    
let strategy = studies.Trading.strategyWithTrailingStop false StockPositionType.Long 0.1m

let outcomesOfInterest = studies.Trading.runTrades priceFunc signalsOfInterest [strategy] |> Async.RunSynchronously

outcomesOfInterest
|> Seq.iter ( fun outcome -> 
    outcome |> outcomeDescription
    printfn ""
)

In [ ]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Types.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.NumberOfTrades)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    outputToConsole $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    outputToConsole $"mapped keys and values for {label}, charting..."
    
    values |> List.zip keys |> List.iter (fun (l,v) -> outputToConsole $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    outputToConsole ""
)

In [ ]:
// let's take only the trades where EV is positive
outputToConsole "Positive EVs only, sorted by EV descending"

tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary outputToConsole false s)

In [ ]:
let benchmarkStrategyName = "All, Sell"

//tradeSummaries |> List.map _.StrategyName |> List.distinct |> List.iter outputToConsole
let benchmarkStrategy = tradeSummaries |> List.find (fun s -> s.StrategyName = benchmarkStrategyName)

outputSummary outputToConsole false benchmarkStrategy

In [ ]:
let newHighsWithPriceBelow20 = outcomes |> Seq.filter (fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28 && o.OpenPrice < o.Sma20)

let count = newHighsWithPriceBelow20 |> Seq.length

//printfn "Found %i" count
//newHighsWithPriceBelow20 |> Seq.take 10 |> Seq.iter outcomeDescription

In [ ]:
let topLoserPriceAbove20 = outcomes |> Seq.filter (fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 30 && o.OpenPrice > o.Sma20 && o.Strategy = "Sell")

let count = topLoserPriceAbove20 |> Seq.length

//printfn "Found %i" count
//topLoserPriceAbove20 |> Seq.take 30 |> Seq.iter outcomeDescription